In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
train_path = '/content/train.csv'
test_path = '/content/test.csv'
train_WineData = pd.read_csv(train_path)
test_WineData = pd.read_csv(test_path)
print('TRAIN DATA')
train_WineData.head()

TRAIN DATA


,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,NaN,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,NaN,Andrew Peace,Chardonnay
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,NaN,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,NaN,Renato Ratti,Nebbiolo
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,NaN,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,NaN,Château du Cèdre,Malbec


In [3]:
print('TEST DATA')
test_WineData.head()

TEST DATA


,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery
0,@paulgwine,US,Boedecker Cellars 2011 Athena Pinot Noir (Will...,Nicely differentiated from the companion Stewa...,Athena,88,35.0,Oregon,Willamette Valley,Willamette Valley,Boedecker Cellars
1,@wineschach,Argentina,Mendoza Vineyards 2012 Gran Reserva by Richard...,"Charred, smoky, herbal aromas of blackberry tr...",Gran Reserva by Richard Bonvin,90,60.0,Mendoza Province,Mendoza,NaN,Mendoza Vineyards
2,@vboone,US,Prime 2013 Chardonnay (Coombsville),"Slightly sour and funky in earth, this is a re...",NaN,87,38.0,California,Coombsville,Napa,Prime
3,@wineschach,Argentina,Bodega Cuarto Dominio 2012 Chento Vineyard Sel...,"This concentrated, midnight-black Malbec deliv...",Chento Vineyard Selection,91,20.0,Mendoza Province,Mendoza,NaN,Bodega Cuarto Dominio
4,@kerinokeefe,Italy,SassodiSole 2012 Brunello di Montalcino,"Earthy aromas suggesting grilled porcini, leat...",NaN,90,49.0,Tuscany,Brunello di Montalcino,NaN,SassodiSole


In [4]:
train_data_columns = list(train_WineData.columns)
test_data_columns = list(test_WineData.columns)
print('columns in training data')
print(train_data_columns)
print('')
print('columns in testing data')
print(test_data_columns)

columns in training data
['user_name', 'country', 'review_title', 'review_description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'winery', 'variety']

columns in testing data
['user_name', 'country', 'review_title', 'review_description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'winery']


In [0]:
# Dropping features useless for classification
train_WineData = train_WineData.drop(['user_name','review_title','review_description','designation','region_2'],axis=1)
test_WineData = test_WineData.drop(['user_name','review_title','review_description','designation','region_2'],axis=1)
# Dropping rows that contains null values
train_WineData = train_WineData.dropna(subset = ['region_1','price'],inplace = False)
test_WineData = test_WineData.dropna(subset = ['region_1','price'],inplace = False)

In [6]:
print('TRAIN DATA INFO')
train_WineData.info()

TRAIN DATA INFO
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65199 entries, 0 to 82656
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country   65199 non-null  object 
 1   points    65199 non-null  int64  
 2   price     65199 non-null  float64
 3   province  65199 non-null  object 
 4   region_1  65199 non-null  object 
 5   winery    65199 non-null  object 
 6   variety   65199 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 4.0+ MB


In [7]:
print('TEST DATA INFO')
test_WineData.info()

TEST DATA INFO
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16203 entries, 0 to 20664
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country   16203 non-null  object 
 1   points    16203 non-null  int64  
 2   price     16203 non-null  float64
 3   province  16203 non-null  object 
 4   region_1  16203 non-null  object 
 5   winery    16203 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 886.1+ KB


In [8]:
print('Train Data Head')
train_WineData.head(10)

Train Data Head


,country,points,price,province,region_1,winery,variety
0,Australia,83,10.0,Australia Other,South Eastern Australia,Andrew Peace,Chardonnay
1,US,89,15.0,Washington,Columbia Valley (WA),North by Northwest,Red Blend
2,Italy,94,80.0,Piedmont,Barolo,Renato Ratti,Nebbiolo
3,France,87,22.0,Southwest France,Bergerac Sec,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,France,88,33.0,France Other,Vin de Liqueur,Château du Cèdre,Malbec
5,Argentina,85,13.0,Other,San Juan,Andean,Cabernet Sauvignon
6,US,91,45.0,Washington,Columbia Valley (WA),Gordon Estate,Red Blend
7,US,90,50.0,California,Sonoma County,Rock Wall,Zinfandel
8,US,91,48.0,Oregon,McMinnville,Elizabeth Chambers,Pinot Noir
9,Italy,86,31.0,Tuscany,Chianti Classico,Mannucci Droandi,Red Blend


In [9]:
print('Test Data Head')
test_WineData.head(10)

Test Data Head


,country,points,price,province,region_1,winery
0,US,88,35.0,Oregon,Willamette Valley,Boedecker Cellars
1,Argentina,90,60.0,Mendoza Province,Mendoza,Mendoza Vineyards
2,US,87,38.0,California,Coombsville,Prime
3,Argentina,91,20.0,Mendoza Province,Mendoza,Bodega Cuarto Dominio
4,Italy,90,49.0,Tuscany,Brunello di Montalcino,SassodiSole
5,US,82,9.0,Missouri,Missouri,Mount Pleasant Winery
7,US,88,26.0,California,Dry Creek Valley,Dry Creek Vineyard
9,Italy,88,24.0,Lombardy,Franciacorta,Contadi Castaldi
10,US,87,25.0,Oregon,Willamette Valley,Spindrift Cellars
11,France,88,35.0,Champagne,Champagne,Eric Taillet


In [0]:
# seperating out unique categorical values for useful categorical features

train_countries = train_WineData['country'].unique()
test_countries = test_WineData['country'].unique()
countries = (np.concatenate([train_countries,test_countries], axis = 0))
countries = np.unique(countries)

train_provinces = train_WineData['province'].unique()
test_provinces = test_WineData['province'].unique()
provinces = (np.concatenate([train_provinces,test_provinces], axis = 0))
provinces = np.unique(provinces)

train_region_1s = train_WineData['region_1'].unique()
test_region_1s = test_WineData['region_1'].unique()
region_1s = (np.concatenate([train_region_1s,test_region_1s], axis = 0))
region_1s = np.unique(region_1s)

train_wineries = train_WineData['winery'].unique()
test_wineries = test_WineData['winery'].unique()
wineries = (np.concatenate([train_wineries,test_wineries], axis = 0))
wineries = np.unique(wineries)

varieties = train_WineData['variety'].unique()
label = train_WineData['variety']
features = test_WineData.columns

classes_dict = {
    'country': countries,
    'province': provinces,
    'region_1': region_1s,
    'winery': wineries,
    'variety': varieties,
    'features': list(features)
}


In [11]:
print('Number of unique countries :', len(countries))
print('Number of unique provinces :', len(provinces))
print('Number of unique regions :', len(region_1s))
print('Number of unique wineries :', len(wineries))
print('Number of unique wine varieties',len(varieties))

Number of unique countries : 7
Number of unique provinces : 63
Number of unique regions : 1029
Number of unique wineries : 12150
Number of unique wine varieties 26


In [0]:
def Get_sparse_column(column_data, classes):
    label_encoder = LabelEncoder()
    label_encoder.fit(classes)
    sparse_column_data = label_encoder.transform(column_data)
    sparse_column_data = sparse_column_data.reshape((len(sparse_column_data),1))
    return sparse_column_data


In [13]:
X_train_sparse = pd.concat([train_WineData['points'], train_WineData['price']],axis=1)
X_test_sparse = pd.concat([test_WineData['points'], test_WineData['price']],axis=1)
print('X_train_sparse')
print(X_train_sparse.head())
print('X_test_sparse')
print(X_test_sparse.head())
# converting to numpy array
X_train_sparse = np.array(X_train_sparse)
X_test_sparse = np.array(X_test_sparse)
print('======================================')
print('')
print('After converting from Dataframe to numpy array')
print(X_train_sparse[:5])
print('')
print(X_test_sparse[:5])

X_train_sparse
   points  price
0      83   10.0
1      89   15.0
2      94   80.0
3      87   22.0
4      88   33.0
X_test_sparse
   points  price
0      88   35.0
1      90   60.0
2      87   38.0
3      91   20.0
4      90   49.0

After converting from Dataframe to numpy array
[[83. 10.]
 [89. 15.]
 [94. 80.]
 [87. 22.]
 [88. 33.]]

[[88. 35.]
 [90. 60.]
 [87. 38.]
 [91. 20.]
 [90. 49.]]


In [14]:
categorical_columns = ['country','province','region_1','winery']
for column_name in categorical_columns:
    
    train_column_data = train_WineData[column_name]
    test_column_data = test_WineData[column_name]
    
    classes = classes_dict[column_name]
    
    train_sparse_column = Get_sparse_column(train_column_data,classes)
    test_sparse_column = Get_sparse_column(test_column_data,classes)
    
    X_train_sparse = np.concatenate((X_train_sparse ,train_sparse_column),axis = 1)
    X_test_sparse = np.concatenate((X_test_sparse ,test_sparse_column),axis = 1)
    print(X_train_sparse.shape[1], X_test_sparse.shape[1])
    

3 3
4 4
5 5
6 6


In [15]:
Y_train_sparse = Get_sparse_column(label,varieties)
print('X_train_sparse.shape = ',X_train_sparse.shape)
print('Y_train_sparse.shape = ',Y_train_sparse.shape)
print('')
print('X_test_sparse.shape = ',X_test_sparse.shape)

X_train_sparse.shape =  (65199, 6)
Y_train_sparse.shape =  (65199, 1)

X_test_sparse.shape =  (16203, 6)


In [16]:
print(X_train_sparse[:10])
print('')
print(Y_train_sparse[:10])
print('')
print(X_test_sparse[:10])

[[8.300e+01 1.000e+01 1.000e+00 3.000e+00 8.560e+02 3.540e+02]
 [8.900e+01 1.500e+01 6.000e+00 6.100e+01 2.390e+02 8.630e+03]
 [9.400e+01 8.000e+01 4.000e+00 4.500e+01 6.200e+01 9.627e+03]
 [8.700e+01 2.200e+01 3.000e+00 5.200e+01 8.300e+01 4.951e+03]
 [8.800e+01 3.300e+01 3.000e+00 1.500e+01 9.610e+02 3.446e+03]
 [8.500e+01 1.300e+01 0.000e+00 4.300e+01 7.870e+02 3.430e+02]
 [9.100e+01 4.500e+01 6.000e+00 6.100e+01 2.390e+02 6.040e+03]
 [9.000e+01 5.000e+01 6.000e+00 8.000e+00 8.380e+02 9.795e+03]
 [9.100e+01 4.800e+01 6.000e+00 4.200e+01 5.210e+02 5.218e+03]
 [8.600e+01 3.100e+01 4.000e+00 5.700e+01 1.820e+02 7.844e+03]]

[[ 5]
 [15]
 [11]
 [ 1]
 [ 9]
 [ 3]
 [15]
 [25]
 [14]
 [15]]

[[8.8000e+01 3.5000e+01 6.0000e+00 4.2000e+01 1.0170e+03 1.1300e+03]
 [9.0000e+01 6.0000e+01 0.0000e+00 2.8000e+01 5.3000e+02 8.1550e+03]
 [8.7000e+01 3.8000e+01 6.0000e+00 8.0000e+00 2.4800e+02 9.3680e+03]
 [9.1000e+01 2.0000e+01 0.0000e+00 2.8000e+01 5.3000e+02 1.0180e+03]
 [9.0000e+01 4.9000e+01 4.0000

In [0]:
np.save('X_train_sparse.npy',X_train_sparse)
np.save('Y_train_sparse.npy',Y_train_sparse)
np.save('X_test_sparse.npy',X_test_sparse)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
X_train,X_test,Y_train,Y_test = train_test_split(X_train_sparse,Y_train_sparse,test_size = 0.3)
classifier1 = RandomForestClassifier(n_estimators = 110)
classifier1.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=110,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
test_predictions = classifier1.predict(X_test)
train_predictions = classifier1.predict(X_train)
print('Train Accuracy :', accuracy_score(Y_train,train_predictions))
print('Test Accuracy :', accuracy_score(Y_test,test_predictions))

Train Accuracy : 0.9733780319463616
Test Accuracy : 0.5826175869120654


In [0]:
predictions = classifier1.predict(X_test_sparse)
import pandas as pd 
pd.DataFrame(predictions).to_csv("predictions.csv")